In [ ]:
# Asthetics
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Basic
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import os
import random
from tqdm.autonotebook import tqdm
import string
import re
from functools import partial
from pprint import pprint

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from wordcloud import WordCloud, STOPWORDS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, pickle, time, gc, copy, sys
import yaml
import pandas_profiling as pdp
import warnings
from tqdm import tqdm
import re
import json
from transformers import RobertaTokenizer, RobertaModel
import nltk

tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
pd.set_option('display.max_columns', 100) # 表示できる列数

In [ ]:
!nvidia-smi

In [ ]:
!ls -la /kaggle/input/download-spacy

In [ ]:
!pip install -U spacy[cuda110] --no-index --find-links /kaggle/input/download-spacy/spacy_cuda110/ # gpu
# !pip install -U spacy --no-index --find-links /kaggle/input/download-spacy/spacy/ # cpu

In [ ]:
!pip install /kaggle/input/download-spacy/en_core_web_lg-3.0.0-py3-none-any.whl/en_core_web_lg-3.0.0-py3-none-any.whl
# roberta
!pip install spacy-transformers --no-index --find-links /kaggle/input/download-spacy/spacy-transformers
!pip install /kaggle/input/download-spacy/en_core_web_trf-3.0.0-py3-none-any.whl/en_core_web_trf-3.0.0-py3-none-any.whl

In [ ]:
import spacy
print(spacy.prefer_gpu())
# nlp = spacy.load('en_core_web_lg') # CPU optimized, not transformer
# nlp = spacy.load('en_core_web_trf') # RoBERTa
# finetuned_ner_nlp = spacy.load("../input/spacy-ner-aug-lexeme/model-best") # lexeme table + data augmentation using the gov data
finetuned_ner_nlp = spacy.load("../input/spacynerlexeme0620/model-best") # added lexeme table

In [ ]:
def read_append_return(filename, train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data
    
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def calc_score(y_true, y_pred, beta=0.5):
    y_true = y_true.split('|')
    y_pred = y_pred.split('|')
#     print(y_true)
#     print(y_pred)
    TP = 0
    FP = 0
    FN = 0
    for i in range(len(y_pred)):
        FP += 1
        for j in range(len(y_true)):
            if jaccard(y_true[j], y_pred[i])>=0.5:
                FP -= 1
                break
    for i in range(len(y_true)):
        FN += 1
        for j in range(len(y_pred)):
            if jaccard(y_true[i], y_pred[j])>=0.5:
                FN -= 1
                TP += 1
                break
    return TP, FP, FN

def detect_duplicated(x):
    for i in range(len(df_train_reduced)):
        if x==df_train_reduced['text'][i]:
            return df_train_reduced['Id'][i]
    return 'no dup'

def pickle_save(path, df):
    with open(path, 'wb') as f:
        pickle.dump(df, f)

def pickle_load(path):
    with open(path, 'rb') as f:
        df = pickle.load(f)
    return df

def ri(df):
    return df.reset_index(drop=True)


In [ ]:
def det_acronym_ver1(text, keywords, TH_LEN_CHAR = 3):
    ans = []
    # text = re.sub("-", " ", text)
    words = text.split()
    for i, word in enumerate(words):
        if word[0]!='(' or word[-1]!=')': continue # (XXX)の形でなければスルー
        acronym_cand = word[1:-1]
        if acronym_cand.lower()==acronym_cand: continue # 大文字が一つもないならスルー
        len_acronym_cand = len(acronym_cand)
        if len_acronym_cand<TH_LEN_CHAR: continue # 3文字以下ならスルー
        acronym_cand_lower = acronym_cand.lower()
        acronym_cand_reverse = acronym_cand_lower[::-1]
        words_cand = words[np.clip(i-len_acronym_cand*3, 0, i):i] # (XXX)の前の数単語を抽出
        words_cand = ' '.join(words_cand).strip()
        words_cand = re.sub('[^A-Za-z0-9]+', ' ', words_cand).strip()
        words_cand = words_cand.split(" ")
        words_cand_reverse = words_cand[::-1]
        idx = 0 # acronym_candの文字index
#         print("words_cand_reverse[0]", words_cand_reverse[0])
        for j, word in enumerate(words_cand_reverse):
            if idx==len_acronym_cand:
                break
            if len(word)==0:
                continue
            if word[0].lower()==acronym_cand_reverse[idx]:
                if idx==len_acronym_cand-1: # 1文字目を検出
                    idx_start = j # dataset名の1単語目のindex
                idx += 1
        if idx==len_acronym_cand:
            words_reverse = words_cand_reverse[:idx_start+1]
            dataset = ' '.join(words_reverse[::-1]).strip().lower()
            if detect_keywords(dataset, keywords):
                acronym = acronym_cand
#                 print("Acronym: {}".format(acronym))
#                 print("Dataset: {}".format(dataset))
                ans.append([acronym_cand, dataset])
        return ans

In [ ]:
def chunk_text0(text, CHUNK_SIZE = 512): # textを文に分けkeywordを含むものだけを抽出
    sentences_i = nltk.tokenize.sent_tokenize(text)
    sentences_i2 = []
    for j, sentence in enumerate(sentences_i):
        for keyword in keywords:
            if keyword in sentence.lower():
                sentences_i2.append(sentence)
    token_chunks = []
    for j, sentence in enumerate(sentences_i2):
        token_j = tokenizer(sentence)['input_ids'][1:-1]
        num_chunk = int(np.ceil(len(token_j)/CHUNK_SIZE))
        for k in range(num_chunk):
            token_chunks.append(token_j[k*CHUNK_SIZE:(k+1)*CHUNK_SIZE])
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]
    return text_chunks

def chunk_text(text, CHUNK_SIZE = 512, keywords=[]): # textを文に分けkeywordを含むものだけを抽出
    sentences_i = nltk.tokenize.sent_tokenize(text)
    sentences_i2 = []
    for j, sentence in enumerate(sentences_i):
        for keyword in keywords:
            if keyword in sentence.lower():
                sentences_i2.append(sentence)
    token_chunks = []
    for j, sentence in enumerate(sentences_i2):
        token_j = tokenizer(sentence)['input_ids'][1:-1]
        num_chunk = int(np.ceil(len(token_j)/CHUNK_SIZE))
        for k in range(num_chunk):
            token_chunks.append(token_j[k*CHUNK_SIZE:(k+1)*CHUNK_SIZE])
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]
    text_chunks2 = []
    for j, chunk in enumerate(text_chunks):
        for keyword in keywords:
            if keyword in chunk.lower():
                text_chunks2.append(chunk)
    return text_chunks2

def get_df2(sentence_list):
    df_ner = []
    for i, sent in enumerate(sentence_list):
        doc_tmp = nlp(sent) # sentenceで区切ってバッチ化して処理される
    #     print(doc_tmp)
        df_tmps = [str(chunk) for chunk in doc_tmp.noun_chunks]
        if len(df_tmps)>0:
            df_tmps = pd.DataFrame(df_tmps, columns=['chunk'])
            df_tmps['sentence_idx'] = i
            df_ner.append(df_tmps)
    if len(df_ner)>0:
        df_ner = pd.concat(df_ner).reset_index(drop=True)
    else:
        df_ner = pd.DataFrame(columns=['chunk'])
    return df_ner

def monitor_loop(i, len_data, start, verbose):
    if (i+1)%verbose==0:
        print("{}/{}, sec: {:.1f}".format(i+1, len_data, time.time()-starttime))

In [ ]:
def check_no_capital(x):
    try:
        if x[1:]==x[1:].lower():
            return True
        return False
    except:
        return False

def delete_the(x):
    try:
        x_split = x.split()
        if x_split[0]=='the':
            return ' '.join(x_split[1:])
        return x
    except:
        return x

def check_including_acronym(x):
    try:
        x_split = re.split("[ ']" , x)
        for word in x_split:
            if word.upper()==word and word.lower()!=word: # 大文字だけの単語があるか
                return True
        return False
    except:
        return True    
    
def get_match(x, ref_labels, threshold=0.5):
    for label in ref_labels:
        if jaccard(x, label)>=threshold:
            return label
    return 'no_match'

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    if len(a)==0 or len(b)==0:
        return 0
    return float(len(c)) / (len(a) + len(b) - len(c))

def delete_keywords(x, ref):
    try:
        x_split = x.split()
        x_new = []
        for item in x_split:
            if item.lower() not in ref:
                x_new.append(item)
        x_new = ' '.join(x_new)
        return x_new
    except:
        return ''


def check_head_capital(x):
    try:
        if len(x)==0:
            return False
        return x[0]==x[0].upper() and x[0]!=x[0].lower() 
    except:
        return False

def detect_black(x, ref):
    try:
        for item in ref:
            if item in x:
                return True
        return False
    except:
        return True
    
def detect_label(x, ref_label, ref_target):
    try:
        predict = []
        for i in range(len(ref_label)):
            if ref_label[i] in x:
                predict.append(ref_target[i])
        return np.array(predict, np.int64)
    except:
        return np.array([], np.int64)
    
def get_label(x, ref_label, ref_target):
    try:
        predict = []
        for i in range(len(x)):
            predict.append(ref_label[ref_target==x[i]][0])
        predict = np.unique(predict).tolist()
        predict = '|'.join(predict)
        return predict
    except:
        return ''
    
def detect_acronym(x, ref_label, ref_acronym, ref_target, th=[1,2]):
    try:
        ans = []
        for i in range(len(ref_label)):
            if x.count(ref_label[i])>=th[0] and x.count(ref_acronym[i])>=th[1]:
                ans.append(ref_target[i])
    #             print(i)
        return ans
    except:
        return []
    
def detect_keywords(x, ref):
    try:
        for keyword in ref:
            if keyword in x:
                return True
        return False
    except:
        return False

In [ ]:
keywords = [
    'study',
    'studies',
    'data',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',   
]

keywords2 = [
    'study',
    'studies',
    'data',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',
    'the',
]
keywords3 = [
    'study',
    'studies',
    'dataset',
    'database',
    'survey',
    'panel',
    'census',
    'cohort',
    'longitudinal',
    'registry',
]
keywords4 = [
    'system',
    'center',
    'centre',
    'committee',
    'documentation',
    'entry',
    'assimilation',
    'explorer',
    'regulation',
    'portal',
    'format',
    'data science',
    'analysis',
    'management',
    'agreement',
    'branch',
    'acquisition',
    'request',
    'task force',
    'program',
    'operator',
    'office',
    'data view',
    'data language',
    'mission',
    'alliance',
    'data model',
    'data structure',
    'corporation',
]

white_list = [
    'ipeds',
]

keywords5 = keywords + ['of', 'the', 'national', 'education']

ng_list = [
    'national longitudinal survey',
    'education longitudinal survey',
    'census bureau',
    'data appendix',
    'data file user',
    'supplementary data',
    'data supplement',
    'major field of study'
]
black_list = [
    'USGS',
    'GWAS',
    'ECLS',
    'DAS',
    'NCDC',
    'NDBC',
    'UDS',
    'GTD',
    'ISC',
    'DGP',
    'EDC',
    'FDA',
    'TSE',
    'DEA',
    'CDA',
    'IDB',
    'NGDC',
    'JODC',
    'EDM',
    'FADN',
    'LRD',
    'DBDM',
    'DMC',
    'WSC',
    ###count4##
]

# Data loading

In [ ]:
df_train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
print(df_train.shape)
df_train.head()

In [ ]:
df_train_reduced = pd.read_csv("../input/coleridge-ext/df_train_reduced.csv")
print(df_train_reduced.shape)
df_train_reduced.head()

In [ ]:
df_test = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
print(df_test.shape)
df_test.head()

In [ ]:
test_files_path = "../input/coleridgeinitiative-show-us-the-data/test"
df_test['text'] = df_test['Id'].progress_apply(lambda x: read_append_return(x, train_files_path=test_files_path))
df_test['clean_text'] = df_test['text'].progress_apply(clean_text)
df_test.head()

In [ ]:
# find train data in test data
df_test['dup_id'] = df_test['text'].progress_apply(lambda x: detect_duplicated(x))
df_test['dup'] = df_test['dup_id']!='no dup'
if len(df_test)==4:
    df_test['dup'][0] = False
df_test.head()

In [ ]:
df_traintest = pd.concat([df_train_reduced, df_test[df_test['dup']==False]]).reset_index(drop=True)
print(df_traintest.shape)
df_traintest.head()

In [ ]:
if len(df_test)==4:
    df_test_reduced = df_train_reduced.iloc[:1000][['Id', 'text', 'clean_text']].copy()
    df_test_reduced['text'].iloc[-1] = " "
    df_test_reduced['clean_text'].iloc[-1] = " "
else:
    df_test_reduced = df_test[df_test['dup']==False].reset_index(drop=True).iloc[:][['Id', 'text', 'clean_text']]

print(df_test_reduced.shape)
df_test_reduced.head()

In [ ]:
if len(df_test)==4:
    df_traintest = df_train_reduced.iloc[:100].copy()
#     df_traintest['text'].iloc[-1] = ""
#     df_traintest['clean_text'].iloc[-1] = ""
#     df_traintest = df_train_reduced
print(df_traintest.shape)
df_traintest.head()

# train label

In [ ]:
df_label = df_train['cleaned_label'].value_counts().reset_index()
df_label.columns = ['cleaned_label', 'count']
df_label['-count'] = -df_label['count']
df_label = df_label.sort_values(['-count', 'cleaned_label']).reset_index(drop=True)
df_label['target'] = np.arange(len(df_label))
df_label['cleaned_label'] = df_label['cleaned_label'].apply(clean_text)

df_tmp1 = df_train[['dataset_label', 'cleaned_label', 'dataset_title']]
df_tmp2 = df_train[['dataset_title', 'cleaned_label', 'dataset_title']]
df_tmp3 = df_train[['cleaned_label', 'cleaned_label', 'dataset_title']]
df_tmp1.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp2.columns = ['label', 'cleaned_label', 'dataset_title']
df_tmp3.columns = ['label', 'cleaned_label', 'dataset_title']
df_label2 = pd.concat([df_tmp1, df_tmp2, df_tmp3])

df_label2['label'] = df_label2['label'].apply(lambda x: x.lower())
df_label2['cleaned_label_my'] = df_label2['label'].apply(clean_text)
df_label2['cleaned_label'] = df_label2['cleaned_label'].apply(clean_text)
df_label2 = df_label2[df_label2['label'].duplicated()==False].reset_index(drop=True)
df_label2 = pd.merge(df_label2, df_label, on='cleaned_label', how='left')
print(df_label2.shape)
print(np.sum(pd.isna(df_label2['target'])))
# print(df_label2[df_label2['target']==1])
df_label2 = df_label2.sort_values(['-count', 'label']).reset_index(drop=True)
df_label2['target_relabeled'] = df_label2['target']
df_label2['target_relabeled'][df_label2['cleaned_label']!=df_label2['cleaned_label_my']] =\
    np.arange(np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my'])) + df_label2['target'].max()+1
print("not match: ", np.sum(df_label2['cleaned_label']!=df_label2['cleaned_label_my']))
print("unique cleaned_label_my: ", len(df_label2['cleaned_label_my'].unique()))
print(df_label2['target'].max())
print(df_label2['target_relabeled'].max())
df_label_train = df_label2
df_label_train.head()

In [ ]:
df_label_train['label-keywords'] = df_label_train['cleaned_label_my'].apply(
    lambda x: delete_keywords(x, keywords5))
df_label_train.head()

In [ ]:
df_train['target'] = df_train['cleaned_label'].progress_apply(lambda x: df_label2['target'][df_label2['cleaned_label']==x.strip()].values[0])
df_tmp = df_train.groupby('Id')['target'].agg(lambda x: list(x)).reset_index()
df_tmp.columns = ['Id', 'targets']
df_tmp['targets'] = df_tmp['targets'].apply(lambda x: np.array([x]).flatten())
df_train_reduced2 = pd.merge(df_train_reduced, df_tmp, on='Id', how='left')
df_train_reduced2.head()

# train label acronym

In [ ]:
acronym_list = [
    ['National Education Longitudinal Study', 'nels'],
#     ['NOAA Tide Gauge', 'nan'],
    ['Sea, Lake, and Overland Surges from Hurricanes', 'slosh'],
    ['Coastal Change Analysis Program', 'c-cap'],
    ['Aging Integrated Database (AGID)', 'agid'],
    ["Alzheimer's Disease Neuroimaging Initiative (ADNI)", 'adni'],
    ['Baltimore Longitudinal Study of Aging (BLSA)', 'blsa'],
    ['Agricultural Resource Management Survey', 'arms'],
    ['Beginning Postsecondary Student', 'bps'],
    ["The National Institute on Aging Genetics of Alzheimer's Disease Data Storage Site (NIAGADS)", 'niagads'],
    ['Common Core of Data', 'ccd'],
#     ['Survey of Industrial Research and Development', 'nan'],
    ['Baccalaureate and Beyond', 'b&b'],
    ['International Best Track Archive for Climate Stewardship', 'IBTrACS'],
    ['National Teacher and Principal Survey', 'ntps'],
    ['Higher Education Research and Development Survey', 'herd'],
    ['Survey of Earned Doctorates', 'sed'],
    ['School Survey on Crime and Safety', 'ssocs'],
    ['World Ocean Database', 'wod'],
    ['Program for the International Assessment of Adult Competencies', 'piaac'],
    ['Early Childhood Longitudinal Study', 'ecls'],
#     ['Survey of Graduate Students and Postdoctorates in Science and Engineering', 'nan'],
    ['Trends in International Mathematics and Science Study', 'timss'],
    ['Education Longitudinal Study', 'els'],
    ['Optimum Interpolation Sea Surface Temperature', 'oisst'],
    ['National Assessment of Education Progress', 'naep'],
    ['High School Longitudinal Study', 'hsls'],
    ['Survey of Doctorate Recipients', 'sdr'],
    ['Rural-Urban Continuum Codes', 'rucc'],
#     ['Survey of Science and Engineering Research Facilities', 'nan'],
#     ['FFRDC Research and Development Survey', 'nan'],
#     ['Survey of State Government Research and Development', 'nan'],
    ['Advanced National Seismic System (ANSS) Comprehensive Catalog (ComCat)', 'comcat'],
#     ['Census of Agriculture', 'nan'],
    ['North American Breeding Bird Survey (BBS)', 'bbs'],
    ['COVID-19 Open Research Dataset (CORD-19)', 'cord-19'],
    ['Complexity Science Hub COVID-19 Control Strategies List (CCCSL)', 'cccsl'],
#     ['Our World in Data COVID-19 dataset', 'nan'],
    ['COVID-19 Precision Medicine Analytics Platform Registry (JH-CROWN)', 'jh-crown'],
    ['Characterizing Health Associated Risks, and Your Baseline Disease In SARS-COV-2 (CHARYBDIS)', 'charybdis'],
#     ['COVID-19 Deaths data', 'nan'],
#     ['SARS-CoV-2 genome sequence', 'nan'],
#     ['COVID-19 Image Data Collection', 'nan'],
    ['RSNA International COVID-19 Open Radiology Database (RICORD)', 'ricord'],
#     ['CAS COVID-19 antiviral candidate compounds dataset', 'nan'],
]
df_label_train_acronym = pd.DataFrame(acronym_list, columns=['dataset_title', 'acronym'])
df_label_train_acronym['acronym_clean'] = df_label_train_acronym['acronym'].apply(clean_text)
df_label_train_acronym['target'] = np.arange(len(df_label_train_acronym))+df_label_train['target'].max()+1
df_label_train_acronym

In [ ]:
TH_LEN_SELF = 4
df_label_train_acronym2 = pd.merge(df_label_train_acronym, df_label_train[['cleaned_label_my', 'dataset_title']], on='dataset_title', how='left')
print(df_label_train_acronym2.shape)
df_label_train_acronym2['base'] = df_label_train_acronym2['cleaned_label_my']
df_label_train_acronym2['label'] = df_label_train_acronym2['acronym_clean']
df_label_train_acronym2['len'] = df_label_train_acronym2['acronym_clean'].apply(lambda x: len(x))
df_label_train_acronym2 = ri(df_label_train_acronym2[df_label_train_acronym2['len']>=TH_LEN_SELF])
print(df_label_train_acronym2.shape)
df_label_train_acronym2.head()

In [ ]:
df_label_train_ref = df_label_train.copy()
print(df_label_train_ref['dataset_title'].unique().shape)
df_agg = df_label_train_ref.groupby('dataset_title')['target_relabeled'].agg(min).reset_index()
df_agg.columns = ['dataset_title', 'target_group']
df_label_train_ref = pd.merge(df_label_train_ref, df_agg, on='dataset_title', how='left')
df_label_train_ref = df_label_train_ref[df_label_train_ref['dataset_title']!='National Education Longitudinal Study']
df_label_train_ref = df_label_train_ref[df_label_train_ref['dataset_title']!='Education Longitudinal Study']
df_label_train_ref = ri(df_label_train_ref)
print(df_label_train_ref['target_group'].unique().shape)
print(df_label_train_ref.shape)
df_label_train_ref.head()

# Detect Acronym ver1

In [ ]:
df_test_reduced['det_acronym'] = df_test_reduced['text'].progress_apply(lambda x: det_acronym_ver1(x, keywords))
df_test_reduced.head()

In [ ]:
tmp = []
for i in range(len(df_test_reduced)):
    if df_test_reduced['det_acronym'][i] is not None:
        tmp += df_test_reduced['det_acronym'][i]
df_label_acronym_ver1_test = pd.DataFrame(tmp)
df_label_acronym_ver1_test.columns = ['acronym', 'base']
df_label_acronym_ver1_test['base_acronym'] = df_label_acronym_ver1_test['base']+"|"+df_label_acronym_ver1_test['acronym']

df_label_acronym_ver1_test_tmp = df_label_acronym_ver1_test[df_label_acronym_ver1_test['base_acronym'].duplicated()==False]
df_agg = df_label_acronym_ver1_test.groupby('base_acronym')['base'].agg(len).reset_index()
cols_tmp = df_agg.columns.tolist()
cols_tmp[-1] = 'count_BA'
df_agg.columns = cols_tmp
df_label_acronym_ver1_test = pd.merge(df_label_acronym_ver1_test_tmp, df_agg, on='base_acronym', how='left')

print(df_label_acronym_ver1_test.shape)
# BA unique 325
df_label_acronym_ver1_test.head(30)

In [ ]:
df_label_acronym_ver1_train = pd.read_csv("../input/coleridge-ext/df_label_acronym_ver1_all_210619_02.csv")
print(df_label_acronym_ver1_train.shape)
df_label_acronym_ver1_train.head()

In [ ]:
if len(df_test)==4:
    df_label_acronym_ver1_test['acronym'].iloc[-1] = 'DUMMY'
    df_label_acronym_ver1_test['base'].iloc[-1] = 'dummy'
    df_label_acronym_ver1_test['base_acronym'].iloc[-1] = 'dummy|DUMMY'
df_label_acronym_ver1_test['test'] = True
df_tmp = df_label_acronym_ver1_train.copy()
df_tmp['count_BA_train'] = df_tmp['count_BA']
df_label_acronym_ver1_test_tmp = pd.merge(df_label_acronym_ver1_test, df_tmp[['base_acronym', 'count_BA_train']], on='base_acronym', how='left')
print(df_label_acronym_ver1_test_tmp.shape)
df_label_acronym_ver1_train_tmp = pd.merge(df_label_acronym_ver1_train, df_label_acronym_ver1_test[['base_acronym', 'test']], on='base_acronym', how='left')
df_label_acronym_ver1_train_tmp = ri(df_label_acronym_ver1_train_tmp[df_label_acronym_ver1_train_tmp['test']!=True])
print(df_label_acronym_ver1_train_tmp.shape)
df_label_acronym_ver1 = ri(pd.concat([df_label_acronym_ver1_test_tmp, df_label_acronym_ver1_train_tmp], axis=0))
print(df_label_acronym_ver1.shape)
df_label_acronym_ver1['cleaned_label'] = df_label_acronym_ver1['base'].apply(clean_text)
df_label_acronym_ver1['target'] = np.arange(len(df_label_acronym_ver1))\
    +df_label_train_acronym['target'].max()+1
df_label_acronym_ver1.head()

In [ ]:
df_label_acronym_ver1['match_train'] = df_label_acronym_ver1['base'].apply(
    get_match, ref_labels=df_label_train['cleaned_label_my'])
df_label_acronym_ver1['label-keywords'] = df_label_acronym_ver1['base'].apply(
    lambda x: delete_keywords(x, keywords5))
df_label_acronym_ver1['len_label-keywords'] = df_label_acronym_ver1['label-keywords'].apply(
    lambda x: len(x.split()))
df_label_acronym_ver1['match_train-keywords'] = df_label_acronym_ver1['label-keywords'].apply(lambda x:
    get_match(x, ref_labels=df_label_train['label-keywords'], threshold=0.75))
df_label_acronym_ver1['acronym_clean'] = df_label_acronym_ver1['acronym'].apply(clean_text)
df_label_acronym_ver1['match_train_acronym'] = df_label_acronym_ver1['acronym_clean'].apply(
    lambda x: x in df_label_train_acronym['acronym_clean'].tolist()
)
df_label_acronym_ver1['keyword3'] = df_label_acronym_ver1['base'].apply(lambda x: detect_keywords(x.lower(), keywords3))
df_label_acronym_ver1['keyword4'] = df_label_acronym_ver1['base'].apply(lambda x: detect_keywords(x.lower(), keywords4))
df_label_acronym_ver1['black'] = df_label_acronym_ver1['acronym'].apply(lambda x: x.upper() in black_list)
df_label_acronym_ver1['white'] = df_label_acronym_ver1['acronym_clean'].apply(lambda x: x in white_list)

df_label_acronym_ver1.head()

In [ ]:
idx_tmp = (df_label_acronym_ver1['len_label-keywords']>0)
idx_tmp = idx_tmp & (df_label_acronym_ver1['match_train-keywords']=='no_match')
idx_tmp = idx_tmp & (df_label_acronym_ver1['match_train_acronym']==False)
# idx_tmp = idx_tmp & (df_label_acronym_ver1['black']==False)
idx_tmp = idx_tmp & ((df_label_acronym_ver1['keyword3'])|(df_label_acronym_ver1['keyword4']==False))
idx_tmp = idx_tmp | (df_label_acronym_ver1['white'])
idx_tmp = idx_tmp & (df_label_acronym_ver1['cleaned_label'].duplicated()==False)
df_label_acronym_ver1_2 =ri(df_label_acronym_ver1[idx_tmp])
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_2.shape)
print(df_label_acronym_ver1_2[df_label_acronym_ver1_2['cleaned_label'].duplicated()==False].shape)
df_label_acronym_ver1_2.head(30)

In [ ]:
df_traintest['det_acronym'] = df_traintest['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_2['cleaned_label'], 
                   df_label_acronym_ver1_2['acronym_clean'], df_label_acronym_ver1_2['target'])
)
df_traintest.head()

In [ ]:
tmp = df_traintest['det_acronym'].values
tmp = np.concatenate(tmp)

df_label_acronym_ver1_2['count'] = df_label_acronym_ver1_2['target'].apply(lambda x: np.sum(tmp==x))
df_label_acronym_ver1_2 = df_label_acronym_ver1_2.sort_values('count', ascending=False)
df_label_acronym_ver1_2

In [ ]:
if len(df_test)==4:
    TH_COUNT = 0 # 1 is best for private
else:
    TH_COUNT = 1
idx_tmp =  (df_label_acronym_ver1_2['count']>=TH_COUNT)
df_label_acronym_ver1_black =ri(df_label_acronym_ver1_2[idx_tmp])
idx_tmp =  (df_label_acronym_ver1_2['count']>=TH_COUNT)
idx_tmp = idx_tmp & (df_label_acronym_ver1_2['black']==False)
df_label_acronym_ver1_3 =ri(df_label_acronym_ver1_2[idx_tmp])
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_2.shape)
print(df_label_acronym_ver1_black.shape)
print(df_label_acronym_ver1_3.shape)
df_label_acronym_ver1_3.head(30)

# Detect Acronym ver1 self

In [ ]:
TH_COUNT = 1 # 1 is best for private
df_label_acronym_ver1_self = df_label_acronym_ver1_3.drop('target', axis=1)
df_tmp = ri(df_label_acronym_ver1_self[df_label_acronym_ver1_self['acronym_clean'].duplicated()==False])
df_tmp['target'] = np.arange(len(df_tmp)) + df_label_acronym_ver1_3['target'].max()+1
df_label_acronym_ver1_self = pd.merge(
    df_label_acronym_ver1_self, df_tmp[['acronym_clean', 'target']], on='acronym_clean', how='left')
print(df_label_acronym_ver1.shape)
print(df_label_acronym_ver1_self.shape)
print(df_label_acronym_ver1_self[df_label_acronym_ver1_self['cleaned_label'].duplicated()==False].shape)
df_label_acronym_ver1_self.head(30)

# Govt

In [ ]:
df_govt = pd.read_csv("../input/coleridge-ext/datasets_govt.csv")
print(df_govt.shape)
df_govt.head()

In [ ]:
df_govt['keyword'] = df_govt['title'].progress_apply(lambda x: detect_keywords(x, keywords))
idx_tmp = df_govt['keyword']
df_govt2 = ri(df_govt[df_govt['keyword']])
df_govt2['cleaned_label'] = df_govt2['title'].progress_apply(clean_text)
df_govt2['match_train'] = df_govt2['cleaned_label'].progress_apply(
    get_match, ref_labels=df_label_train['cleaned_label_my'])

idx_tmp = df_govt2['keyword']
print(df_govt2[idx_tmp].shape)
df_govt2[idx_tmp].head()

In [ ]:
df_govt2['len'] = df_govt2['cleaned_label'].progress_apply(lambda x: len(x.split()))
idx_tmp = df_govt2['match_train']=='no_match'
idx_tmp = df_govt2['len']>=3
df_govt3 = ri(df_govt2[idx_tmp])
print(df_govt2[idx_tmp].shape)
df_govt2[idx_tmp].head()

In [ ]:
df_govt3['match_ver1'] = df_govt2['cleaned_label'].progress_apply(
    get_match, ref_labels=df_label_acronym_ver1_3['cleaned_label'])
idx_tmp = df_govt3['match_ver1']=='no_match'
idx_tmp = idx_tmp & (df_govt3['match_train']=='no_match')
df_govt4 = ri(df_govt3[idx_tmp])
print(df_govt4.shape)
df_govt4.head()

In [ ]:
df_govt4['target'] = np.arange(len(df_govt4)) + df_label_acronym_ver1_self['target'].max()+1
df_govt4.head()

In [ ]:
idx_tmp = df_govt4['len']<=10
df_govt5 = df_govt4[idx_tmp]
df_govt5 = ri(df_govt5.sort_values('len'))
print(df_govt5.shape)
df_govt5.head()

In [ ]:
def func(x, ref):
    index_tmp = x['index']
    if index_tmp==0:
        return 'no_match'
    ref = ref[:index_tmp-1]
    for item in ref:
        if jaccard(x['cleaned_label'], item)>=0.5:
            return item
    return 'no_match'
ref_list = df_govt5['cleaned_label'].tolist()
df_govt5['index'] = np.arange(len(df_govt5))
if len(df_test)==4:
    df_govt5 = df_govt5.iloc[:1000]
df_govt5['match_in_self'] = df_govt5.progress_apply(lambda x: func(x, ref_list), axis=1)

In [ ]:
idx_tmp = df_govt5['match_in_self']=='no_match'
df_govt6 = ri(df_govt5[idx_tmp])
print(df_govt6.shape)
df_govt6.head()

In [ ]:
df_traintest['det_govt'] = df_traintest['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_govt6['cleaned_label'].values, 
                 ref_target=df_govt6['target'].values))

In [ ]:
tmp = df_traintest['det_govt'].values
tmp = np.concatenate(tmp)

df_govt6['count'] = df_govt6['target'].apply(lambda x: np.sum(tmp==x))
df_govt6 = ri(df_govt6.sort_values('count', ascending=False))
df_govt6

In [ ]:
TH_COUNT = 2
idx_tmp = df_govt6['count']>=TH_COUNT
idx_tmp = idx_tmp & (df_govt6['match_train']=='no_match')
df_govt7 = ri(df_govt6[idx_tmp])
print(df_govt7.shape)
df_govt7.head(30)

# NER
det1, trainの表記ゆれ検出  
trainはdataset名でグルーピングする
els, nelsは検出しない。

In [ ]:
tokenizer = pickle_load("../input/coleridge-ext/tokenizer_210613_01.pkl")

In [ ]:
df_label_ref = df_label_acronym_ver1_3[['cleaned_label', 'target']].copy()
df_tmp = df_label_train_ref[['cleaned_label_my', 'target_group']]
df_tmp.columns = ['cleaned_label', 'target']
df_label_ref = ri(pd.concat([df_label_ref, df_tmp], axis=0))
print(df_label_ref.shape)
df_label_ref.head()

In [ ]:
df_test_reduced['det_ref'] = df_test_reduced['clean_text'].progress_apply(lambda x: 
    detect_label(x, df_label_ref['cleaned_label'], df_label_ref['target'].values)
)
df_test_reduced['len_det'] = df_test_reduced['det_ref'].apply(len)
print(df_test_reduced[df_test_reduced['len_det']>0].shape) # 340,6
df_test_reduced[df_test_reduced['len_det']>0].head(30)

In [ ]:
def get_include(x, ref):
    try:
        for item in ref:
            if item in x:
                return item
        return 'no_include'
    except:
        return 'no_include'
    
def get_include2(x, ref):
    try:
        for item in ref:
            if x in item:
                return item
        return 'no_include'
    except:
        return 'no_include'
    
nels_els_list = [
    'national education longitudinal study',
    'education longitudinal study',
]

In [ ]:
%%time
i = 41
BATCH_SIZE = 64
def get_ner_pred(x):
    try:
        text = x['text']
        sentence_list = chunk_text(text, keywords=keywords) # keyword含む文のみのchunkを作成
        det_ref = x['det_ref']
        labels_det_ref = []
        for target in det_ref:
            labels_det_ref.append(df_label_ref['cleaned_label'][df_label_ref['target']==target].values[0])
        labels_det_ref = np.unique(labels_det_ref).tolist()

        num_batch = int(np.ceil(len(sentence_list)/BATCH_SIZE))
        naun_chunks = []
        for j in range(num_batch):
            batch = sentence_list[j*BATCH_SIZE:(j+1)*BATCH_SIZE]
            for k, sent in enumerate(batch):
                naun_chunks_k = list(finetuned_ner_nlp(sent).ents) # sentenceで区切ってバッチ化して処理される
                naun_chunks += naun_chunks_k
        naun_chunks = [str(chunk) for chunk in naun_chunks]
        naun_chunks2 = np.unique(naun_chunks)
        df_chunks = pd.DataFrame(naun_chunks2, columns=['chunk'])
        df_chunks['chunk-the'] = df_chunks['chunk'].apply(delete_the)
        df_chunks['cleaned_label'] = df_chunks['chunk-the'].apply(clean_text)
        df_chunks['len'] = df_chunks['cleaned_label'].apply(lambda x: len(x.split()))
        df_chunks['det_already'] = df_chunks['cleaned_label'].apply(lambda x: x in labels_det_ref)
        df_chunks['match_ref'] = df_chunks['cleaned_label'].apply(
            lambda x: get_match(x, ref_labels=df_label_ref['cleaned_label'], threshold=0.5)
        )
        df_chunks['match_nels_els'] = df_chunks['cleaned_label'].apply(
            lambda x: get_match(x, ref_labels=nels_els_list, threshold=0.5)
        )
        df_chunks['include_ref'] = df_chunks['cleaned_label'].apply(
            lambda x: get_include(x, ref=df_label_ref['cleaned_label'])
        )
        
        df_chunks['include_ref2'] = df_chunks['cleaned_label'].apply(
            lambda x: get_include2(x, ref=df_label_ref['cleaned_label'])
        )
        idx_tmp = df_chunks['cleaned_label'].duplicated()==False
        idx_tmp = idx_tmp & (df_chunks['len']>=3)
        idx_tmp = idx_tmp & (df_chunks['det_already']==False)
        idx_tmp = idx_tmp & (df_chunks['match_ref']!='no_match')
        idx_tmp = idx_tmp & (df_chunks['match_nels_els']=='no_match')
        idx_tmp = idx_tmp & (df_chunks['include_ref']=='no_include')
        idx_tmp = idx_tmp & (df_chunks['include_ref2']=='no_include')
        df_chunks2 = ri(df_chunks[idx_tmp])

        pred_ner = "|".join(df_chunks2['cleaned_label'].values.tolist())
#         pred_ner = 'A'
        return pred_ner
    except:
        return ''

df_test_reduced['pred_ner'] = ""
df_test_reduced['pred_ner'].iloc[:20] = df_test_reduced.iloc[:20].progress_apply(get_ner_pred, axis=1)
df_test_reduced.iloc[10:20]

In [ ]:
"""
ver3
10 joint center for political studies
14 national study of postgraduate faculty

"""
df_test_reduced[df_test_reduced['pred_ner']!='']

In [ ]:
df_label_train_ref['cleaned_label'][df_label_train_ref['target_group']==3]

In [ ]:
label_tmp = 'baltimore longitudinal study on aging'
for i in range(len(df_label_ref)):
    label_i = df_label_ref['cleaned_label'][i]
    if jaccard(label_i, label_tmp)>=0.5:
        print(label_i)

# Test prediction

In [ ]:
def pred_dup(x):
    df_tmp = df_train_reduced2[df_train_reduced2['text']==x]
    if len(df_tmp)>0:
        label_list = df_tmp['targets'].values[0]
    else:
        label_list = np.zeros(0, np.int64)
    return label_list

df_test['det_dup'] = df_test['text'].apply(lambda x: pred_dup(x))
df_test.head()

In [ ]:
df_test['det_train'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_label_train['label'].values, 
                 ref_target=df_label_train['target_relabeled'].values))

In [ ]:
df_test['det_train_acronym'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_train_acronym2['base'], 
                   df_label_train_acronym2['acronym_clean'], 
                   df_label_train_acronym2['target'], th=[1,2])
)
df_test.head()

In [ ]:
df_test['det_govt'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x.lower(), ref_label=df_govt7['cleaned_label'].values, 
                 ref_target=df_govt7['target'].values))

In [ ]:
df_test['det_acronym'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_3['cleaned_label'], 
                   df_label_acronym_ver1_3['acronym_clean'], df_label_acronym_ver1_3['target'], th=[1,0])
)
df_test.head()

In [ ]:
df_test['det_acronym_self'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_acronym(x, df_label_acronym_ver1_self['cleaned_label'], 
                   df_label_acronym_ver1_self['acronym_clean'], df_label_acronym_ver1_self['target'], th=[1,1])
)
df_test.head()

In [ ]:
df_test['det_ref'] = df_test['clean_text'].progress_apply(lambda x: 
    detect_label(x, df_label_ref['cleaned_label'], df_label_ref['target'].values)
)

In [ ]:
df_test['pred_ner'] = df_test.progress_apply(get_ner_pred, axis=1)
df_test.head()

In [ ]:
df_test['pred_det'] = df_test.progress_apply(lambda x: 
    np.sort(np.unique(np.concatenate([
        x['det_acronym_self'],
        x['det_acronym'],
        x['det_govt'],
#         x['det_dup'],
#         x['det_train'],
#         x['det_train_acronym'],
    ]))).astype(np.int64), axis=1
)
df_test.head()

In [ ]:
df_tmp1 = df_label_acronym_ver1_self[['acronym_clean', 'target']]
df_tmp1.columns = ['label', 'target']
df_tmp2 = df_label_acronym_ver1_3[['cleaned_label', 'target']]
df_tmp2.columns = ['label', 'target']
df_tmp3 = df_govt7[['cleaned_label', 'target']]
df_tmp3.columns = ['label', 'target']
df_tmp4 = df_label_train[['cleaned_label_my', 'target']]
df_tmp4.columns = ['label', 'target']
df_tmp5 = df_label_train_acronym2[['acronym_clean', 'target']]
df_tmp5.columns = ['label', 'target']
df_label_all = pd.concat([df_tmp1, df_tmp2, df_tmp3, df_tmp4, df_tmp5]).reset_index(drop=True)
df_label_all = ri(df_label_all[df_label_all['target'].duplicated()==False])
df_label_all.tail()

In [ ]:
df_test['PredictionString'] = df_test['pred_det'].progress_apply(lambda x: 
    get_label(x, df_label_all['label'].values, df_label_all['target'].values)
)
df_test['PredictionString'] = df_test['PredictionString'] + '|' + df_test['pred_ner']
df_test['PredictionString'] = df_test['PredictionString'].apply(lambda x: x.strip('|'))
df_test.head()

In [ ]:
df_sub = df_test[['Id', 'PredictionString']]
df_sub.to_csv("submission.csv", index=None)
df_sub.head()

In [ ]:
for i in range(len(df_sub.iloc[:10])):
    print(df_sub['PredictionString'][i])

In [ ]:
for i in range(len(df_sub.iloc[:10])):
    pred_list = df_sub['PredictionString'][i].split('|')
#     print(pred_list, len(pred_list))
    if pred_list[0]!='':
        for j in range(len(pred_list)):
            print("{:4d}: {}".format(i, pred_list[j]))